In [ ]:
sample_orders=20000
rules = ["HUV_0", "HUV_1"]

## Init

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 

import random
myseed = 13
random.seed(myseed)

import pandas as pd
import numpy as np
import es
import es.runner.scenario_runner
from es.runner.view_models import show_cube_grid, SpectrumViewModel, show_spectrum_grid
import es.elections
import es.utils
from es.data.netflix_data import *
import bokeh
import bokeh.plotting
from bokeh.io import output_notebook
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 40)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
%%time
idata = es.data.netflix_data.NetflixData(es.data_config.data_netflix_parquet)
iframes = es.elections.InstacartDataFrames.from_data(idata)

## Parameters

In [ ]:
PARAMS = es.utils.dotdict(dict(
        exp_name='tmp',
#         k=10,
        k=3,
        searchterms=['^The Dirty Dozen$'],
        filtering=dict(sample_orders=sample_orders),
        rules=rules, # prefix sa. means 'use simulated annealing'
        anneal_settings= {'tmax': 8100.0, 'tmin': 2.6, 'steps': 2000, 'updates': 10}, # anneal_settings={'minutes': 0.5},
        scorer_metrics=['tfidfto2'],
        scorer_funcs = [es.scorers.AgentSpreadScoringStrategy.from_found_df_sorted_nosearchterm]
    ))

## Runner

In [ ]:
%%time
runner = es.runner.scenario_runner.ScenarioRunner(iframes, PARAMS)
exp_folder = runner.run(verbose=1)

## Visualisation

In [ ]:
%%time
es.runner.view_models.show_spectrum_grid(SpectrumViewModel(), PARAMS, exp_folder, plot_side=500, plot_height=50, max_div_value=150)

# Analysis

In [ ]:
from es import *
u = es.electionutils.ElectionUtils(iframes)

In [ ]:
display(iframes.df_products)
# display(iframes.df_orders)
# display(iframes.df_order_products)

## SQL

In [ ]:
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

products = u.product_search
sql(f"select * from products where product_name like 'Star%' and order_count > 1 order by order_count desc")